I am going to use the following metric functions
1. closeness
2. degree
3. betweenness

In [1]:
import preprocessing as ps
import numpy as np
import networkx as nx
import pickle
import matplotlib.pyplot as plt
import random
random.seed(42)
import csv
import algos as al
import metrics as me

In [2]:
# create the project networks
list_1 = [('DM', 'T'), ('DB', 'T'), ('AI', 'T')]
list_2 = [('DM', 'T'), ('DB', 'T'), ('AI', 'T'), ('DM', 'DB'), ('DM', 'AI'), ('DB', 'AI')]
list_3 = [('DM', 'T'), ('DB', 'T'), ('AI', 'DB')]

star_proj = ps.createProjectNetwork(list_1)
full_proj = ps.createProjectNetwork(list_2)
chain_proj = ps.createProjectNetwork(list_3)

main_network = pickle.load(open(f"./networks/new_subnets/20_nodes.pkl", 'rb'))
project_net = chain_proj
beta = None

In [3]:
import time

obj_max_1 = 0.0
best_set_1 = set()

metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
print("Influential First")

influential_nodes = al.get_top_node_from_each_group(network_main.copy(), project_net, metric_fn)
start_time = time.time()
for node in influential_nodes:
    subset, comm_eff = al.Greedy(network_main.copy(), project_net, node, metric_fn=metric_fn, beta=beta)
    if comm_eff > obj_max_1:
        obj_max_1 = comm_eff
        best_set_1 = subset

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

total_inf = sum(al.leader_eff(network_main.copy(), project_net, metric_fn, node, beta=beta) for node in best_set_1)
print("Coordinators communication efficiency", total_inf)
# print(f"Objective value : {obj_max_1}")

print("Intra-team ranking")
for node in list(best_set_1):
    print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.intra_team_rank(network_main, metric_fn, node)}")

# print("\n")

# print("Inter-team ranking")
# for node in list(best_set_1):
#     print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.inter_team_rank(network_main, project_net, metric_fn, node)}")

Influential First
Execution time: 4.625817775726318 seconds
Coordinators communication efficiency 573.1199849724558
Intra-team ranking
Team :AI, Node: Joris M. Mooij, Rank: 1
Team :T, Node: Bogdan S. Chlebus, Rank: 16
Team :DB, Node: Stefan Krompass, Rank: 1
Team :DM, Node: Qiang Yang, Rank: 1


In [4]:
obj_max_2 = 0.0
best_set_2 = set()

metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
print("Greedy Algorithm")

start_time = time.time()
for node in network_main.nodes():
    subset, comm_eff = al.Greedy(network_main.copy(), project_net, node, metric_fn=metric_fn, beta=beta)
    if comm_eff > obj_max_2:
        obj_max_2 = comm_eff
        best_set_2 = subset

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

total_inf = sum(al.leader_eff(network_main.copy(), project_net, metric_fn, node, beta=beta) for node in best_set_2)
print("Coordinators communication efficiency", total_inf)
# print(f"Objective value : {obj_max_2}")

print("Intra-team ranking")
for node in list(best_set_2):
    print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.intra_team_rank(network_main, metric_fn, node)}")

# print("\n")

# print("Inter-team ranking")
# for node in list(best_set_2):
#     print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.inter_team_rank(network_main, project_net, metric_fn, node)}")

Greedy Algorithm
Execution time: 120.71787571907043 seconds
Coordinators communication efficiency 573.1199849724558
Intra-team ranking
Team :AI, Node: Joris M. Mooij, Rank: 1
Team :T, Node: Bogdan S. Chlebus, Rank: 16
Team :DB, Node: Stefan Krompass, Rank: 1
Team :DM, Node: Qiang Yang, Rank: 1


In [5]:
# Using Influence Team Strategy
metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
al.inteam_influence_only(network_main, project_net, metric_fn)

Intra-team ranking
Team :T, Node: Kenneth W. Regan, Rank: 1
Team :AI, Node: Joris M. Mooij, Rank: 1
Team :DB, Node: Stefan Krompass, Rank: 1
Team :DM, Node: Qiang Yang, Rank: 1




564.1673742806807

In [6]:
# Using Random Strategy
num_iter = 1000
metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
al.randomMonteCarlo(network_main, project_net, metric_fn, num_iter)

441.78